In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def indicators(data):
    ####  CCI
    # Calculate the Typical Price
    data['Typical Price'] = (data['HIGH'] + data['LOW'] + data['CLOSE']) / 3
    
    # Define the number of days to use in the SMA and Mean Deviation calculations
    n = 34
    
    # Calculate the Simple Moving Average (SMA)
    data['SMA'] = data['Typical Price'].rolling(n).mean()
    
    # Calculate the Mean Deviation
    data['Mean Deviation'] = np.abs(data['Typical Price'] - data['SMA']).rolling(n).mean()
    
    # Calculate the Commodity Channel Index (CCI)
    data['CCI'] = (data['Typical Price'] - data['SMA']) / (0.015 * data['Mean Deviation'])
    
    # Print the last 10 rows of the data with the CCI column
    # print(data.tail(10))
    data = data.drop_duplicates()
    
    print('CCI:',len(data))
    
    
    
    ### RSI
    # Set time period for RSI calculation
    period = 14
    
    # Calculate price difference
    delta = data['CLOSE'].diff()
    
    # Create dataframe to store RSI values
    rsi_df = pd.DataFrame()
    
    # Calculate gain and loss
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gain and loss for time period
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Add RSI values to dataframe
    rsi_df['TIMESTAMP'] = data['TIMESTAMP']
    rsi_df['RSI'] = rsi
    
    rsi_df = rsi_df.drop_duplicates()
    print('RSI:',len(rsi_df))
    
    ### MACD
    
    # Set time periods for EMA calculation
    short_period = 12
    long_period = 26
    signal_period = 9
    
    # Calculate short and long period EMAs
    short_ema = data['CLOSE'].ewm(span=short_period, adjust=False).mean()
    long_ema = data['CLOSE'].ewm(span=long_period, adjust=False).mean()
    
    # Calculate MACD line
    macd_line = short_ema - long_ema
    
    # Calculate signal line
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    
    # Calculate MACD histogram
    macd_hist = macd_line - signal_line
    
    # Create dataframe to store MACD values
    macd_df = pd.DataFrame()
    
    # Add MACD values to dataframe
    macd_df['TIMESTAMP'] = data['TIMESTAMP']
    macd_df['MACD'] = macd_line
    macd_df['Signal'] = signal_line
    macd_df['Histogram'] = macd_hist
    
    # Display dataframe
    print('MACD:',len(macd_df))
    
    # Merge CCI and RSI
    rsi_cci = data.merge(rsi_df,on ='TIMESTAMP',how= 'inner')
    rsi_cci_macd = rsi_cci.merge(macd_df,on ='TIMESTAMP',how= 'inner')
    
    print('Combined:',len(rsi_cci_macd))
    
    return rsi_cci_macd

In [3]:
'''
This function extracts the Stocks mentioned above and puts them as a separate csv
'''
def extarct_Stocks(stocks_to_extract_list, columns_to_extract, csv_directory):
    '''
    Input:
        stocks_to_extract_list -> List of all the stocks you want to extract
        columns_to_extract -> The four columns we are interested in
        csv_directory -> Where the Bhavcopy csv files are stored
    Output:
        It stores the csv of the stocks in the same directory as the program
    '''
    combined_df = pd.DataFrame()
    for stocks_to_extract in stocks_to_extract_list:

        result_df = pd.DataFrame()

        for filename in os.listdir(csv_directory):
            if filename.endswith('.csv'):
                temp = os.path.join(csv_directory, filename)
                df = pd.read_csv(temp)

                df_stock = df.loc[df['SYMBOL'].isin([stocks_to_extract]), columns_to_extract]

                result_df = pd.concat([result_df, df_stock])
                
        
        # Load the data from the bhavcopy file
        data = result_df.copy()
        
        
        print('Read:',len(data))
        #Remove reduce
        data= data[data['TOTALTRADES']>=50]

        # Convert the 'TIMESTAMP' column to datetime objects
        data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format='%d-%b-%Y')
            
        
        # Sort the data by the 'TIMESTAMP' column in ascending order
        data = data.sort_values('TIMESTAMP')
        
        data = data.drop_duplicates()
 
        result_df_final = indicators(data)
        combined_df = pd.concat([result_df_final, combined_df])
        #new_filename = f'{stocks_to_extract}.csv'
        #result_df_final.to_csv(new_filename, index=False)
    return combined_df

In [4]:
cd C:/UW/WI23/BIOSTAT557/Project/

C:\UW\WI23\BIOSTAT557\Project


In [6]:
if __name__ == "__main__":
    top100 = pd.read_csv('ind_nifty100list.csv')
    stocks_to_extract_list = top100['Symbol'].unique()
    
    #stocks_to_extract_list = ['APOLLOHOSP']
    columns_to_extract = ['TIMESTAMP','SYMBOL','OPEN', 'HIGH', 'LOW', 'CLOSE','LAST','PREVCLOSE','TOTTRDQTY','TOTTRDVAL','TOTALTRADES']
    csv_directory = 'C:/UW/WI23/BIOSTAT557/Project/Bhavcopy'
    #'../Bhavcopy'
    df = extarct_Stocks(stocks_to_extract_list, columns_to_extract, csv_directory)
    
    

ACC
Read: 1735
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
ADANIENT
Read: 1737
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
ADANIGREEN
Read: 1127
CCI: 1121
RSI: 1121
MACD: 1121
Combined: 1121
ADANIPORTS
Read: 1736
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
ATGL
Read: 494
CCI: 488
RSI: 488
MACD: 488
Combined: 488
ADANITRANS
Read: 1737
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
AMBUJACEM
Read: 1737
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
APOLLOHOSP
Read: 1738
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
ASIANPAINT
Read: 1740
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
DMART
Read: 1440
CCI: 1430
RSI: 1430
MACD: 1430
Combined: 1430
AXISBANK
Read: 1742
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
BAJAJ-AUTO
Read: 1735
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
BAJFINANCE
Read: 1738
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
BAJAJFINSV
Read: 1740
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
BAJAJHLDNG
Read: 1729
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
BANDHANB

In [ ]:
df.to_csv('Top100stocks_indicator.csv',index= False)
#df.to_csv('AS.csv',index= False)